In [1]:
import pandas as pd
import os
import ast
from dotenv import load_dotenv
from pymongo import MongoClient

In [2]:
load_dotenv()

True

In [3]:
client = MongoClient(os.getenv("url"))

In [4]:
df = pd.read_csv("eurocup_2020_results.csv")

# Objetivo

Voy a organizar los datos de tal forma que pueda tener un json que cada objeto sea un pais con sus estadisticas, mientras que creare otro json para los jugadores. Finalmente, quizas, cree otro json que los objetos sean las fases de grupo.

- Datos por pais:
    - goles totales a favor/ en contra
    - goles medios por partido a favor/ encontra
    - victorias, empates, derrotas
    - posesion media por partido
    - disparos medios por partido totales
    - diparos medios por partido on target
    - eficacia media: sumatorio (total de disparos on target / total de disparos) / total partidos
    - disparos medios recibidos por partido
    - media de duelos ganados por partido
    - media de tarjetas amarillas / rojas
    - media de cambios por partido
    - rondas de clasificacion

- Datos por jugador:
    - partidos totales jugados
    - titular / suplente
    - minutos totales jugados
    - tarjetas
    - goles
    - numero, nombre
    - media de en que minuto le han cambiado

# 1. Analisis previo, nulls y nan

In [5]:
df

,stage,date,pens,pens_home_score,pens_away_score,team_name_home,team_name_away,team_home_score,team_away_score,possession_home,possession_away,total_shots_home,total_shots_away,shots_on_target_home,shots_on_target_away,duels_won_home,duels_won_away,events_list,lineup_home,lineup_away
0,Final,11.07.2021,True,3,2,Italy,England,1,1,66%,34%,19,6,6,2,53%,47%,"[{'event_team': 'away', 'event_time': "" 2' "", ...","[{'Player_Name': 'Insigne', 'Player_Number': '...","[{'Player_Name': 'Kane', 'Player_Number': '9'}..."
1,Semi-finals,07.07.2021,False,False,False,England,Denmark,2,1,59%,41%,20,6,10,3,50%,50%,"[{'event_team': 'away', 'event_time': "" 30' "",...","[{'Player_Name': 'Kane', 'Player_Number': '9'}...","[{'Player_Name': 'Krogh Damsgaard', 'Player_Nu..."
2,Semi-finals,06.07.2021,True,4,2,Italy,Spain,1,1,29%,71%,7,16,4,5,49%,51%,"[{'event_team': 'away', 'event_time': "" 51' "",...","[{'Player_Name': 'Insigne', 'Player_Number': '...","[{'Player_Name': 'Torres', 'Player_Number': '1..."
3,Quarter-finals,03.07.2021,False,False,False,Ukraine,England,0,4,48%,52%,7,10,2,6,42%,59%,"[{'event_team': 'away', 'event_time': "" 4' "", ...","[{'Player_Name': 'Yaremchuk', 'Player_Number':...","[{'Player_Name': 'Kane', 'Player_Number': '9'}..."
4,Quarter-finals,03.07.2021,False,False,False,Czech Republic,Denmark,1,2,56%,44%,16,11,5,7,48%,52%,"[{'event_team': 'away', 'event_time': "" 5' "", ...","[{'Player_Name': 'Schick', 'Player_Number': '1...","[{'Player_Name': 'Dolberg', 'Player_Number': '..."
5,Quarter-finals,02.07.2021,False,False,False,Belgium,Italy,1,2,46%,54%,10,14,4,3,58%,42%,"[{'event_team': 'away', 'event_time': "" 13' "",...","[{'Player_Name': 'Lukaku', 'Player_Number': '9...","[{'Player_Name': 'Insigne', 'Player_Number': '..."
6,Quarter-finals,02.07.2021,True,1,3,Switzerland,Spain,1,1,27%,73%,8,28,2,10,58%,42%,"[{'event_team': 'away', 'event_time': "" 8' "", ...","[{'Player_Name': 'Seferovic', 'Player_Number':...","[{'Player_Name': 'Sarabia', 'Player_Number': '..."
7,Round of 16,29.06.2021,False,False,False,Sweden,Ukraine,1,2,45%,55%,13,15,3,4,48%,52%,"[{'event_team': 'away', 'event_time': "" 27' "",...","[{'Player_Name': 'Isak', 'Player_Number': '11'...","[{'Player_Name': 'Yaremchuk', 'Player_Number':..."
8,Round of 16,29.06.2021,False,False,False,England,Germany,2,0,45%,55%,5,9,4,3,55%,45%,"[{'event_team': 'home', 'event_time': "" 8' "", ...","[{'Player_Name': 'Sterling', 'Player_Number': ...","[{'Player_Name': 'Werner', 'Player_Number': '1..."
9,Round of 16,28.06.2021,True,4,5,France,Switzerland,3,3,55%,45%,26,12,8,5,55%,45%,"[{'event_team': 'away', 'event_time': "" 15' "",...","[{'Player_Name': 'Mbappe', 'Player_Number': '1...","[{'Player_Name': 'Seferovic', 'Player_Number':..."


In [6]:
df.info()
#no hay nulos
#hay varios int que son str , habra que limpiarlos
#hay booleanos que son str, hay que limpiarlos
#las tres ultimas columnas contienen datos que tenemos que extraer y organizar

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51 entries, 0 to 50
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   stage                 51 non-null     object
 1   date                  51 non-null     object
 2   pens                  51 non-null     bool  
 3   pens_home_score       51 non-null     object
 4   pens_away_score       51 non-null     object
 5   team_name_home        51 non-null     object
 6   team_name_away        51 non-null     object
 7   team_home_score       51 non-null     int64 
 8   team_away_score       51 non-null     int64 
 9   possession_home       51 non-null     object
 10  possession_away       51 non-null     object
 11  total_shots_home      51 non-null     int64 
 12  total_shots_away      51 non-null     int64 
 13  shots_on_target_home  51 non-null     int64 
 14  shots_on_target_away  51 non-null     int64 
 15  duels_won_home        51 non-null     obje

# 2. Limpieza y homogenizacion de los datos

### Primer vistazo

In [7]:
df

,stage,date,pens,pens_home_score,pens_away_score,team_name_home,team_name_away,team_home_score,team_away_score,possession_home,possession_away,total_shots_home,total_shots_away,shots_on_target_home,shots_on_target_away,duels_won_home,duels_won_away,events_list,lineup_home,lineup_away
0,Final,11.07.2021,True,3,2,Italy,England,1,1,66%,34%,19,6,6,2,53%,47%,"[{'event_team': 'away', 'event_time': "" 2' "", ...","[{'Player_Name': 'Insigne', 'Player_Number': '...","[{'Player_Name': 'Kane', 'Player_Number': '9'}..."
1,Semi-finals,07.07.2021,False,False,False,England,Denmark,2,1,59%,41%,20,6,10,3,50%,50%,"[{'event_team': 'away', 'event_time': "" 30' "",...","[{'Player_Name': 'Kane', 'Player_Number': '9'}...","[{'Player_Name': 'Krogh Damsgaard', 'Player_Nu..."
2,Semi-finals,06.07.2021,True,4,2,Italy,Spain,1,1,29%,71%,7,16,4,5,49%,51%,"[{'event_team': 'away', 'event_time': "" 51' "",...","[{'Player_Name': 'Insigne', 'Player_Number': '...","[{'Player_Name': 'Torres', 'Player_Number': '1..."
3,Quarter-finals,03.07.2021,False,False,False,Ukraine,England,0,4,48%,52%,7,10,2,6,42%,59%,"[{'event_team': 'away', 'event_time': "" 4' "", ...","[{'Player_Name': 'Yaremchuk', 'Player_Number':...","[{'Player_Name': 'Kane', 'Player_Number': '9'}..."
4,Quarter-finals,03.07.2021,False,False,False,Czech Republic,Denmark,1,2,56%,44%,16,11,5,7,48%,52%,"[{'event_team': 'away', 'event_time': "" 5' "", ...","[{'Player_Name': 'Schick', 'Player_Number': '1...","[{'Player_Name': 'Dolberg', 'Player_Number': '..."
5,Quarter-finals,02.07.2021,False,False,False,Belgium,Italy,1,2,46%,54%,10,14,4,3,58%,42%,"[{'event_team': 'away', 'event_time': "" 13' "",...","[{'Player_Name': 'Lukaku', 'Player_Number': '9...","[{'Player_Name': 'Insigne', 'Player_Number': '..."
6,Quarter-finals,02.07.2021,True,1,3,Switzerland,Spain,1,1,27%,73%,8,28,2,10,58%,42%,"[{'event_team': 'away', 'event_time': "" 8' "", ...","[{'Player_Name': 'Seferovic', 'Player_Number':...","[{'Player_Name': 'Sarabia', 'Player_Number': '..."
7,Round of 16,29.06.2021,False,False,False,Sweden,Ukraine,1,2,45%,55%,13,15,3,4,48%,52%,"[{'event_team': 'away', 'event_time': "" 27' "",...","[{'Player_Name': 'Isak', 'Player_Number': '11'...","[{'Player_Name': 'Yaremchuk', 'Player_Number':..."
8,Round of 16,29.06.2021,False,False,False,England,Germany,2,0,45%,55%,5,9,4,3,55%,45%,"[{'event_team': 'home', 'event_time': "" 8' "", ...","[{'Player_Name': 'Sterling', 'Player_Number': ...","[{'Player_Name': 'Werner', 'Player_Number': '1..."
9,Round of 16,28.06.2021,True,4,5,France,Switzerland,3,3,55%,45%,26,12,8,5,55%,45%,"[{'event_team': 'away', 'event_time': "" 15' "",...","[{'Player_Name': 'Mbappe', 'Player_Number': '1...","[{'Player_Name': 'Seferovic', 'Player_Number':..."


In [8]:
#funcion para saber si hay mas de un tipo de objeto en una columna
def types(columna):
    sets = set()
    sets.update([type(i) for i in columna])
    return sets

In [9]:
#comprobamos los types de cada columna y ver si en alguna hay mas de uno
for i in df:
    print(f'{i}: {types(df[i])}')

stage: {<class 'str'>}
date: {<class 'str'>}
pens: {<class 'bool'>}
pens_home_score: {<class 'str'>}
pens_away_score: {<class 'str'>}
team_name_home: {<class 'str'>}
team_name_away: {<class 'str'>}
team_home_score: {<class 'int'>}
team_away_score: {<class 'int'>}
possession_home: {<class 'str'>}
possession_away: {<class 'str'>}
total_shots_home: {<class 'int'>}
total_shots_away: {<class 'int'>}
shots_on_target_home: {<class 'int'>}
shots_on_target_away: {<class 'int'>}
duels_won_home: {<class 'str'>}
duels_won_away: {<class 'str'>}
events_list: {<class 'str'>}
lineup_home: {<class 'str'>}
lineup_away: {<class 'str'>}


### Comprobamos los datos de cada columna

#### columnas stage, date y pens

In [10]:
#todas ok

#### columnas pens_home_score y pens_away_score

In [11]:
#es str y necesito pasarlo a bool e ints

In [12]:
#creamos una funcion para cambiar los numeros a enteros y los F/T a bool.
def entero(valor):
    if valor == "False":
        return False
    if valor == "True":
        return True
    if valor.isalpha:
        return int(valor)

In [13]:
#aplicamos la funcion, a las columnas 
df["pens_home_score"] = df["pens_home_score"].apply(entero)
df["pens_away_score"] = df["pens_away_score"].apply(entero)

In [14]:
#compurebo que se han hecho los cambios
print(types(df["pens_home_score"]))
print(types(df["pens_home_score"]))

{<class 'int'>, <class 'bool'>}
{<class 'int'>, <class 'bool'>}


#### col team_name_home	team_name_away	

In [15]:
df["team_name_home"][0] #vemos que los nombres tienen espacios, vamos a eliminarlos

' Italy '

In [16]:
#funcion para eliminar los espacios tanto al principio como al final de un str
def no_spaces(data):
    return data.strip()

In [17]:
df["team_name_home"] = df["team_name_home"].apply(no_spaces)
df["team_name_away"] = df["team_name_away"].apply(no_spaces)
df["stage"]  = df["stage"].apply(no_spaces)

In [18]:
#comprobamos que se han eliminado los espacios
df["team_name_home"][0]

'Italy'

#### col team_home_score	team_away_score

In [19]:
# colunnas ok, todos son enteros

#### col possession_home	possession_away

In [20]:
df["possession_home"][0]

'66%'

In [21]:
#hay que pasarlos a enteros y quitar el %
#el metodo find nos devuelve la posicoin del elemento, por eso hacemos str[str.find()] asi, nos devuelve el str.

def per_int(value):
        return int(value.replace(value[value.find("%")], ""))

In [22]:
df["possession_home"] = df["possession_home"].apply(per_int)
df["possession_away"] = df["possession_away"].apply(per_int)

In [23]:
type(df["possession_home"][0])

numpy.int64

#### col total_shots_home	total_shots_away

In [24]:
#todo ok

#### col shots_on_target_home	shots_on_target_away

In [25]:
#todo ok

#### col duels_won_home	duels_won_away

In [26]:
#igual que con las de posesion, aplicamos la mimsa formula
df["duels_won_home"][0]

'53%'

In [27]:
df["duels_won_home"] = df["duels_won_home"].apply(per_int)
df["duels_won_away"] = df["duels_won_away"].apply(per_int)

In [28]:
type(df["duels_won_home"][0])

numpy.int64

#### col events_list , lineup_home,  lineup_away

In [29]:
#con la funcion ast.literal_eval(), logramos obtener una lista de diccionarios mucho mas limpia y accesible
# ademas, de pasarlo de str a lista

In [30]:
def lst_ok(value):
    return ast.literal_eval(value)

In [31]:
df1 = df

In [32]:
df["events_list"] = df["events_list"].apply(lst_ok)
df["lineup_home"] = df["lineup_home"].apply(lst_ok)
df["lineup_away"] = df["lineup_away"].apply(lst_ok)

##### limpieza de valores anidados

In [33]:
#comprobamos los types dentro de las keys anidadas
#vemos que el event_time tiene dos formatos
s = set()
for i in df["events_list"]:
    for j in i:
        for k in j:
            t = (k, type(j[k]))
            s.add(t)
for i,j in s:
    print(i,j)

event_player <class 'str'>
event_result <class 'str'>
action_player_2 <class 'str'>
event_team <class 'str'>
event_time <class 'bool'>
event_type <class 'str'>
action_player_1 <class 'str'>
event_time <class 'str'>


In [34]:
#limpiamos los espacios
for i in df["events_list"]:
    for j in i:
        for k in j:
            if type(j[k]) != bool:
                j[k] = no_spaces(j[k])

In [35]:
#qitar las comillas y pasrlo a int
def comillas_int(value):
        return int(value.replace(value[value.find("'")], ""))

for i in df1["events_list"]:
    for j in i:
        for k in j:
            if k == 'event_time' and j[k] != False:
                j[k] = comillas_int(j[k])

In [36]:
df["lineup_home"][0] #de momento no veo el porque cambiarlo a int

[{'Player_Name': 'Insigne', 'Player_Number': '10'},
 {'Player_Name': 'Immobile', 'Player_Number': '17'},
 {'Player_Name': 'Chiesa', 'Player_Number': '14'},
 {'Player_Name': 'Verratti', 'Player_Number': '6'},
 {'Player_Name': 'Jorginho', 'Player_Number': '8'},
 {'Player_Name': 'Barella', 'Player_Number': '18'},
 {'Player_Name': 'Emerson', 'Player_Number': '13'},
 {'Player_Name': 'Chiellini', 'Player_Number': '3'},
 {'Player_Name': 'Bonucci', 'Player_Number': '19'},
 {'Player_Name': 'Di Lorenzo', 'Player_Number': '2'},
 {'Player_Name': 'Donnarumma', 'Player_Number': '21'}]

## Crear dataframe de los paises

In [37]:
df_clean = df

### Paises

In [38]:
#creamos una lista con los paises
teams = []
for i in df_clean["team_name_home"].unique():
    teams.append(i)
teams.sort()

In [39]:
#creamos la data para los paises
teams

['Austria',
 'Belgium',
 'Croatia',
 'Czech Republic',
 'Denmark',
 'England',
 'Finland',
 'France',
 'Germany',
 'Hungary',
 'Italy',
 'Netherlands',
 'North Macedonia',
 'Poland',
 'Portugal',
 'Russia',
 'Scotland',
 'Slovakia',
 'Spain',
 'Sweden',
 'Switzerland',
 'Turkey',
 'Ukraine',
 'Wales']

### Partidos totales

In [40]:
matches_total = []
for i in range(len(teams)):
    apariciones = 0
    for j, k in df_clean['team_name_home'].value_counts().iteritems():
        if teams[i] == j:
            apariciones = k
    for l, m in df_clean['team_name_away'].value_counts().iteritems():
        if teams[i] == l:
            apariciones = apariciones + m
    matches_total.append(apariciones)
matches_total

[4, 5, 4, 5, 6, 7, 3, 4, 4, 3, 7, 4, 3, 3, 4, 3, 3, 3, 6, 4, 5, 3, 5, 4]

### Goles

#### goles totales favor / contra

In [41]:
#creamos lista para total goals in favour:
total_goals_favour = []
for i in teams:
    l = df_clean[(df_clean["team_name_home"] == i)]["team_home_score"].sum()
    a = df_clean[(df_clean["team_name_away"] == i)]["team_away_score"].sum()
    total_goals_favour.append(l+a)

In [42]:
#creamos una lista para total goals against
total_goals_against = []
for i in teams:
    l = df_clean[(df_clean["team_name_home"] == i)]["team_away_score"].sum()
    a = df_clean[(df_clean["team_name_away"] == i)]["team_home_score"].sum()
    total_goals_against.append(l+a)

#### goles por partido

In [43]:
#creamos la lista para los goles por partido a favor y en contra
# calculamos las aparicones totales del torneo y dividimos los goles totales entre el total de partidos
goal_mean_favour = []
goal_mean_against = []

for i in range(len(teams)):
    apariciones = 0
    for j, k in df_clean['team_name_home'].value_counts().iteritems():
        if teams[i] == j:
            apariciones = k
            #print(teams[i], "apariciones", apariciones)
    for l, m in df_clean['team_name_away'].value_counts().iteritems():
        if teams[i] == l:
            apariciones = apariciones + m
            #print(teams[i],"aparicones tot", apariciones)
    #print("total goles favor", total_goals_favour[i], "total goles contra", total_goals_against[i])
    goal_mean_favour.append(round(total_goals_favour[i]/(apariciones),2))
    goal_mean_against.append(round(total_goals_against[i]/(apariciones),2))

### Disparos

#### Total Shots, total shots on target

In [44]:
#creamos una lista con los disparos totales de cada pais
total_shots = []
for i in teams:
    l = df_clean[(df_clean["team_name_home"] == i)]["total_shots_home"].sum()
    a = df_clean[(df_clean["team_name_away"] == i)]["total_shots_away"].sum()
    total_shots.append(l+a)

In [45]:
#creamos lista con los disparos totales a puerta de cada pais
total_shots_target = []
for i in teams:
    l = df_clean[(df_clean["team_name_home"] == i)]["shots_on_target_home"].sum()
    a = df_clean[(df_clean["team_name_away"] == i)]["shots_on_target_away"].sum()
    total_shots_target.append(l+a)


#### Media de tiros y de tiros ontarget por partido

In [46]:
#media de tiros on targetg por partido(tot ontarget / tot partidos)
avg_shots_target = []

for i in range(len(teams)):
    apariciones = 0
    for j, k in df_clean['team_name_home'].value_counts().iteritems():
        if teams[i] == j:
            apariciones = k
    for l, m in df_clean['team_name_away'].value_counts().iteritems():
        if teams[i] == l:
            apariciones = apariciones + m
    avg_shots_target.append(round(total_shots_target[i]/apariciones, 2))
    

In [47]:
#media de tiros por partido(tot shots / tot partidos)
avg_shots_tot = []
total_shots = []
for i in range(len(teams)):
    apariciones = 0
    for j, k in df_clean['team_name_home'].value_counts().iteritems():
        if teams[i] == j:
            apariciones = k
    for l, m in df_clean['team_name_away'].value_counts().iteritems():
        if teams[i] == l:
            apariciones = apariciones + m
    
    l = df_clean[(df_clean["team_name_home"] == teams[i])]["total_shots_home"].sum()
    a = df_clean[(df_clean["team_name_away"] == teams[i])]["total_shots_away"].sum()
    
    total_shots.append(l+a)
    
    avg_shots_tot.append(round(total_shots[i]/apariciones, 2))
    

#### Total de disparos recibidos, media de disparos recibidos por partido

In [48]:
#calculamos el total de disparos recibidos
total_shots_against = []

for i in teams:
    l = df_clean[(df_clean["team_name_home"] == i)]["total_shots_away"].sum()
    a = df_clean[(df_clean["team_name_away"] == i)]["total_shots_home"].sum()
    total_shots_against.append(l+a)

In [49]:
#calculamos la media de disparos recibida por partido (total_shots_against / matches_total)

avg_shots_against = []
for i,j in zip(total_shots_against, matches_total):
    avg_shots_against.append(round(i/j, 2))


In [50]:
avg_shots_against

[13.25,
 14.0,
 13.25,
 10.4,
 10.0,
 9.57,
 17.67,
 9.25,
 6.25,
 14.67,
 8.57,
 10.25,
 14.67,
 11.33,
 8.5,
 12.0,
 10.0,
 16.33,
 6.5,
 15.0,
 19.0,
 21.0,
 13.8,
 18.75]

#### Eficacia: total goals favour/total shots target

In [51]:
#creamos una lista con la eficacia que es de los tiros ontragetm cuantos son gol. Prob de que sea gol cuando es ontarget

In [52]:
eficacia = []
for i,j in zip(total_goals_favour,total_shots_target):
    eficacia.append(round(i/j*100,2))


### Posesion

In [53]:
#calculamos la possesion media por partido
pos_avg = []
#calculamos la suma de las posesiones por equipo
pos_sum = []
for i in teams:
    l = df_clean[(df_clean["team_name_home"] == i)]["possession_home"].sum()
    a = df_clean[(df_clean["team_name_away"] == i)]["possession_away"].sum()
    pos_sum.append(l+a)
#calculamos la posesion media por partido
for i,j in zip(pos_sum, matches_total):
    pos_avg.append(round(i/j,2))

### Stage

In [54]:
#Calculamos la ultima fase a la que han llegado cada equipo
last_stage = []
#creamos una funcion que itere sobre el dataframe y nos retorne cuando se cumpla la condicion
def stages(team, a,b,c):#duda de *args
    for i,j,k in zip(a,b,c):
        if team == j or team == k:
            return i
#iteramos sobre teams y creamos la lista correspondiente a la ultima fase que alcanza cada equipo
for n in teams:
    last_stage.append(stages(n, df_clean["stage"], df_clean["team_name_home"], df_clean["team_name_away"]))               

In [55]:
last_stage

['Round of 16',
 'Quarter-finals',
 'Round of 16',
 'Quarter-finals',
 'Semi-finals',
 'Final',
 'Group stage: Matchday 3',
 'Round of 16',
 'Round of 16',
 'Group stage: Matchday 3',
 'Final',
 'Round of 16',
 'Group stage: Matchday 3',
 'Group stage: Matchday 3',
 'Round of 16',
 'Group stage: Matchday 3',
 'Group stage: Matchday 3',
 'Group stage: Matchday 3',
 'Semi-finals',
 'Round of 16',
 'Quarter-finals',
 'Group stage: Matchday 3',
 'Quarter-finals',
 'Round of 16']

### Duelos

#### % total de duelos ganados

In [56]:
#calculamos los duelos ganados en porcentaje. Total de duelos ganados, entre el total de duelos de los partidos que han jugado.
duel_won_by_team = [] #duelos totales ganados por equpo en el torneo
duel_tot_by_team =  [] #douelos totales disputados por equipo en el torneo
duel_win_rate_tot = [] # % de duelos ganados en todo el torneo
for i in range(len(teams)):
    #calculamos los duelos ganados por equipo
    l = df_clean[(df_clean["team_name_home"] == teams[i])]["duels_won_home"].sum()
    a = df_clean[(df_clean["team_name_away"] == teams[i])]["duels_won_away"].sum()
    duel_won_by_team.append(l+a)
    #calculamos los duelos tot disputados por equipo
    l = df_clean[(df_clean["team_name_home"] == teams[i])]["duels_won_home"].sum()
    a = df_clean[(df_clean["team_name_home"] == teams[i])]["duels_won_away"].sum()
    l2 = df_clean[(df_clean["team_name_away"] == teams[i])]["duels_won_home"].sum()
    a2 = df_clean[(df_clean["team_name_away"] == teams[i])]["duels_won_away"].sum()
    duel_tot_by_team.append(l+a+l2+a2)
    
    #calculamos el % de duelos ganados por cada equipo
    duel_win_rate_tot.append(round(duel_won_by_team[i]/duel_tot_by_team[i], 2))

In [57]:
duel_win_rate_tot

[0.51,
 0.52,
 0.5,
 0.48,
 0.52,
 0.52,
 0.47,
 0.54,
 0.46,
 0.44,
 0.49,
 0.53,
 0.48,
 0.54,
 0.55,
 0.51,
 0.49,
 0.5,
 0.52,
 0.44,
 0.53,
 0.46,
 0.48,
 0.47]

### Media de tarjetas por partido

In [58]:
#calculamos las tarjetas amarillas (no contamos las segundas tarjetas amarillas, dado que las contaremos como rojas)

In [60]:
#para ver todos los tipos de eventos
events = set()
for i in df_clean["events_list"]:
    for j in i:
        events.add(j['event_type'])

In [61]:
events

{'Disallowed goal',
 'Goal',
 'Own goal',
 'PK',
 'Penalty',
 'Red card',
 'Second yellow card',
 'Substitution',
 'Yellow card'}

In [67]:
df_clean[(df_clean["team_name_home"] == "Italy")]["events_list"][0]

[{'event_team': 'away',
  'event_time': 2,
  'event_type': 'Goal',
  'action_player_1': 'Luke Shaw',
  'action_player_2': 'Kieran Trippier'},
 {'event_team': 'home',
  'event_time': 47,
  'event_type': 'Yellow card',
  'action_player_1': 'Nicolo Barella'},
 {'event_team': 'home',
  'event_time': 54,
  'event_type': 'Substitution',
  'action_player_1': 'Bryan Cristante',
  'action_player_2': 'Nicolo Barella'},
 {'event_team': 'home',
  'event_time': 55,
  'event_type': 'Yellow card',
  'action_player_1': 'Leonardo Bonucci'},
 {'event_team': 'home',
  'event_time': 55,
  'event_type': 'Substitution',
  'action_player_1': 'Domenico Berardi',
  'action_player_2': 'Ciro Immobile'},
 {'event_team': 'home',
  'event_time': 67,
  'event_type': 'Goal',
  'action_player_1': 'Leonardo Bonucci'},
 {'event_team': 'away',
  'event_time': 70,
  'event_type': 'Substitution',
  'action_player_1': 'Bukayo Saka',
  'action_player_2': 'Kieran Trippier'},
 {'event_team': 'away',
  'event_time': 74,
  'even

#### tarjetas amarillas totales

In [79]:
#calculamos el numero de tarjetas amarillas totales
yellow_cards_tot = []
for i in teams:
    count = 0
    for j in df_clean[(df_clean["team_name_home"] == i)]["events_list"]:
        for k in j:
            if k['event_team'] == "home" and k["event_type"] == "Yellow card":
                count+=1
    for j in df_clean[(df_clean["team_name_away"] == i)]["events_list"]:
        for k in j:
            if k['event_team'] == "away" and k["event_type"] == "Yellow card":
                count+=1
    yellow_cards_tot.append(count)

#### Tarjetas amarillas por partido

In [80]:
#calculamos las tarjetas amarillas por partido
yellow_cards_avg = []
for i,j in zip(yellow_cards_tot,matches_total):
    yellow_cards_avg.append(round(i/j,2))
yellow_cards_avg    

[1.5,
 0.8,
 1.75,
 1.4,
 0.83,
 0.86,
 1.33,
 2.0,
 1.75,
 2.0,
 1.71,
 0.75,
 2.67,
 2.33,
 1.0,
 1.67,
 1.0,
 2.0,
 1.0,
 1.25,
 2.2,
 2.33,
 0.8,
 2.25]

#### Tarjetas rojas

In [81]:
#calculamos el total de tarejetas rojas
red_cards_tot = []
for i in teams:
    count = 0
    for j in df_clean[(df_clean["team_name_home"] == i)]["events_list"]:
        for k in j:
            if k['event_team'] == "home" and k["event_type"] == 'Red card':
                count += 1
            if k['event_team'] == "home" and k["event_type"] == 'Second yellow card':
                count += 1
    for j in df_clean[(df_clean["team_name_away"] == i)]["events_list"]:
        for k in j:
            if k['event_team'] == "away" and k["event_type"] == 'Red card':
                count += 1
            if k['event_team'] == "away" and k["event_type"] == 'Second yellow card':
                count += 1
    red_cards_tot.append(count)

In [82]:
#calculamos las tarjetas rojas por partido
red_cards_avg = []
for i,j in zip(red_cards_tot,matches_total):
    red_cards_avg.append(round(i/j,2))
red_cards_avg 

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.25,
 0.0,
 0.33,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.25,
 0.2,
 0.0,
 0.0,
 0.5]

### Cambios

In [84]:
#calculamos la media de cambios por partido. Primero los cambios totales.
subs_tot = []
for i in teams:
    count = 0
    for j in df_clean[(df_clean["team_name_home"] == i)]["events_list"]:
        for k in j:
            if k['event_team'] == "home" and k["event_type"] == 'Substitution':
                count+=1
    for j in df_clean[(df_clean["team_name_away"] == i)]["events_list"]:
        for k in j:
            if k['event_team'] == "away" and k["event_type"] == 'Substitution':
                count+=1
    subs_tot.append(count)
subs_tot

[19,
 18,
 18,
 25,
 31,
 26,
 13,
 14,
 18,
 12,
 37,
 19,
 14,
 12,
 18,
 13,
 11,
 15,
 32,
 20,
 24,
 14,
 17,
 14]

In [85]:
#calculamos los cambios por partido}
subs_avg = []
for i,j in zip(subs_tot,matches_total):
    subs_avg.append(round(i/j,2))
subs_avg 

[4.75,
 3.6,
 4.5,
 5.0,
 5.17,
 3.71,
 4.33,
 3.5,
 4.5,
 4.0,
 5.29,
 4.75,
 4.67,
 4.0,
 4.5,
 4.33,
 3.67,
 5.0,
 5.33,
 5.0,
 4.8,
 4.67,
 3.4,
 3.5]

In [ ]:
df_clean

In [86]:
euro_df = pd.DataFrame({"Teams": teams,
                        "Stage" : last_stage,
                        "Partidos jugados": matches_total,
                           "Total goals favour": total_goals_favour , 
                           "Total goals against": total_goals_against , 
                           "Goals favour per game": goal_mean_favour , 
                           "Goals against per game": goal_mean_against ,
                           "Accuracy": eficacia,
                        "Shots total": total_shots,
                        "Shots total on target": total_shots_target,
                           "Shots per game": avg_shots_tot,
                           "Shots on target per game": avg_shots_target,
                       "Shots Total against": total_shots_against,
                           "Shots against per game": avg_shots_against,
                       "Possession average": pos_avg,
                       "Yellow cards total":yellow_cards_tot,
                       "Yellow cards per game": yellow_cards_avg,
                       "Red cards total": red_cards_tot,
                       "Red cards per game": red_cards_avg,
                       "Subs per game":subs_avg})
euro_df

,Teams,Stage,Partidos jugados,Total goals favour,Total goals against,Goals favour per game,Goals against per game,Accuracy,Shots total,Shots total on target,Shots per game,Shots on target per game,Shots Total against,Shots against per game,Possession average,Yellow cards total,Yellow cards per game,Red cards total,Red cards per game,Subs per game
0,Austria,Round of 16,4,5,5,1.25,1.25,35.71,53,14,13.25,3.50,53,13.25,53.50,6,1.50,0,0.00,4.75
1,Belgium,Quarter-finals,5,9,3,1.80,0.60,42.86,48,21,9.60,4.20,70,14.00,53.80,4,0.80,0,0.00,3.60
2,Croatia,Round of 16,4,7,8,1.75,2.00,41.18,42,17,10.50,4.25,53,13.25,49.50,7,1.75,0,0.00,4.50
3,Czech Republic,Quarter-finals,5,6,4,1.20,0.80,31.58,56,19,11.20,3.80,52,10.40,48.20,7,1.40,0,0.00,5.00
4,Denmark,Semi-finals,6,12,7,2.00,1.17,30.77,92,39,15.33,6.50,60,10.00,53.33,5,0.83,0,0.00,5.17
5,England,Final,7,11,2,1.57,0.29,39.29,63,28,9.00,4.00,67,9.57,51.00,6,0.86,0,0.00,3.71
6,Finland,Group stage: Matchday 3,3,1,3,0.33,1.00,33.33,19,3,6.33,1.00,53,17.67,36.67,4,1.33,0,0.00,4.33
7,France,Round of 16,4,7,6,1.75,1.50,38.89,56,18,14.00,4.50,37,9.25,53.25,8,2.00,0,0.00,3.50
8,Germany,Round of 16,4,6,7,1.50,1.75,33.33,49,18,12.25,4.50,25,6.25,62.75,7,1.75,0,0.00,4.50
9,Hungary,Group stage: Matchday 3,3,3,6,1.00,2.00,30.00,19,10,6.33,3.33,44,14.67,29.00,6,2.00,0,0.00,4.00


## 3. Creacion del json

In [ ]:
#vamos a crear un json para poder subirlo a mongodb